In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('D:/工作/data/titanic')

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
cat = df.select_dtypes(include='object')

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
enc = OneHotEncoder(categories='auto',handle_unknown='ignore')

In [9]:
cat = cat.fillna('None')

In [14]:
result = enc.fit_transform(cat)

In [31]:
from sklearn.preprocessing import LabelEncoder

In [37]:
label = LabelEncoder()
res = label.fit_transform(cat.iloc[:,2])

In [43]:
label.inverse_transform()

array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649', '239865', '248698', '330923', '113788',
       '349909', '347077', '2631', '19950', '330959', '349216',
       'PC 17601', 'PC 17569', '335677', 'C.A. 24579', 'PC 17604',
       '113789', '2677', 'A./5. 2152', '345764', '2651', '7546', '11668',
       '349253', 'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371',
       '14311', '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572',
       '2926', '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651',
       'CA 2144', '2669', '113572', '36973', '347088', 'PC 17605', '2661',
       'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111',
       'CA 2144', 'S.O.C. 14879', '2680', '1601', '348123', '349208',
       '374746', '248738', '364516', '345767', '345779', '330932',
       '113059',

In [20]:
mapping={'x{}'.format(i):value for i, value in enumerate(cat.columns)}

In [29]:
pattern=re.compile(r'^x\d+')
d = map(lambda x: pattern.sub(mapping[pattern.match(x).group()],x),
        enc.get_feature_names())

In [30]:
list(d)

['Name_Abbing, Mr. Anthony',
 'Name_Abbott, Mr. Rossmore Edward',
 'Name_Abbott, Mrs. Stanton (Rosa Hunt)',
 'Name_Abelson, Mr. Samuel',
 'Name_Abelson, Mrs. Samuel (Hannah Wizosky)',
 'Name_Adahl, Mr. Mauritz Nils Martin',
 'Name_Adams, Mr. John',
 'Name_Ahlin, Mrs. Johan (Johanna Persdotter Larsson)',
 'Name_Aks, Mrs. Sam (Leah Rosen)',
 'Name_Albimona, Mr. Nassef Cassem',
 'Name_Alexander, Mr. William',
 'Name_Alhomaki, Mr. Ilmari Rudolf',
 'Name_Ali, Mr. Ahmed',
 'Name_Ali, Mr. William',
 'Name_Allen, Miss. Elisabeth Walton',
 'Name_Allen, Mr. William Henry',
 'Name_Allison, Master. Hudson Trevor',
 'Name_Allison, Miss. Helen Loraine',
 'Name_Allison, Mrs. Hudson J C (Bessie Waldo Daniels)',
 'Name_Allum, Mr. Owen George',
 'Name_Andersen-Jensen, Miss. Carla Christine Nielsine',
 'Name_Anderson, Mr. Harry',
 'Name_Andersson, Master. Sigvard Harald Elias',
 'Name_Andersson, Miss. Ebba Iris Alfrida',
 'Name_Andersson, Miss. Ellis Anna Maria',
 'Name_Andersson, Miss. Erna Alexandra',


In [49]:
df.mode(numeric_only=True).iloc[0]

PassengerId     1.00
Survived        0.00
Pclass          3.00
Age            24.00
SibSp           0.00
Parch           0.00
Fare            8.05
Name: 0, dtype: float64

In [53]:
a = df.loc[:,['Survived', 'Pclass','Age','SibSp','Parch','Fare']].mode()

In [57]:
df.isnull().any().any()

True

In [58]:
col = ['Survived', 'Pclass','Age','SibSp','Parch','Fare']
df.columns.difference(set(col))

Index(['Cabin', 'Embarked', 'Name', 'PassengerId', 'Sex', 'Ticket'], dtype='object')

In [59]:
import xgboost as xgb

In [60]:
es = xgb.XGBClassifier()

In [61]:
es.get_params()['n_estimators']

100

In [62]:
import lightgbm as lgb
es1 = lgb.LGBMClassifier()
es1.get_params()['n_estimators']

100

In [64]:
es.fit(df.loc[:,col[1:]], df.loc[:,col[0]])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [84]:
ypred = es.predict_proba(df.loc[:,col[1:]], ntree_limit=40)

In [85]:
from sklearn.metrics import roc_curve

In [ ]:
y = df.loc[:,col[0]]
ypred = ypred[:,1]

In [87]:
fpr, tpr, thres = roc_curve(y, ypred)

In [89]:
np.abs(fpr-tpr).max()

0.4793883616144186

In [92]:
from sklearn.metrics import auc, roc_auc_score

In [91]:
auc(fpr, tpr)

0.8212646065680291

In [93]:
roc_auc_score(y, ypred)

0.8212646065680291

In [99]:
category, bins = pd.qcut(ypred, q=10, duplicates='drop', retbins=True)

In [112]:
cate = category.set_categories(bins[:-1], rename=True)

In [116]:
group = y.groupby(cate)

In [120]:
(group.sum()/y.sum()).sort_index(ascending=False).cumsum()

0.732731    0.242690
0.576536    0.415205
0.477955    0.564327
0.401960    0.663743
0.329217    0.763158
0.282369    0.871345
0.213180    0.926901
0.198277    0.970760
0.149845    0.988304
0.052730    1.000000
Name: Survived, dtype: float64

In [117]:
group.size()

0.052730     93
0.149845     91
0.198277     84
0.213180     90
0.282369     93
0.329217    104
0.401960     79
0.477955     82
0.576536     87
0.732731     88
Name: Survived, dtype: int64

In [126]:
columns=['箱','分数','样本数量','关注样本数','区间关注类别比例',
        '累计关注类别比例','区间准确率','累计准确率','尾部准确率',
        '提升','累计提升']
tb = pd.DataFrame(columns=columns,index=bins[:-1][::-1])

In [129]:
tb['样本数量'] = group.size()

In [131]:
tb['分数'] = tb.index

In [138]:
tb['箱'] = range(1,len(tb)+1)
tb['关注样本数'] = group.sum()
tb

,箱,分数,样本数量,关注样本数,区间关注类别比例,累计关注类别比例,区间准确率,累计准确率,尾部准确率,提升,累计提升
0.732731,1,0.732731,88,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.576536,2,0.576536,87,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.477955,3,0.477955,82,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.401960,4,0.401960,79,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.329217,5,0.329217,104,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.282369,6,0.282369,93,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.213180,7,0.213180,90,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.198277,8,0.198277,84,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.149845,9,0.149845,91,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.052730,10,0.052730,93,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
tb.groupby(tb['箱']<=2)['样本数量'].sum()

箱
False    716
True     175
Name: 样本数量, dtype: int64

In [156]:
y1=(ypred>bins[:-1][::-1][1]).astype(int)

In [157]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [158]:
confusion_matrix(y, y1)

array([[516,  33],
       [200, 142]], dtype=int64)

In [160]:
precision_score(y, y1), recall_score(y, y1)

(0.8114285714285714, 0.4152046783625731)

In [197]:
from sklearn import metrics
class ResultShow:
    def get_group(self, y, ypred):
        category, bins = pd.qcut(ypred, q=10, duplicates='drop', retbins=True)
        cate = category.set_categories(bins[:-1], rename=True)
        return cate, bins[:-1][::-1]

    def bin_info(self, y, ypred):
        columns=['箱','分数','样本数量','关注样本数','区间关注类别比例',
        '累计关注类别比例','区间准确率','累计准确率','尾部准确率',
        '提升','累计提升']
        cate, bins = self.get_group(y, ypred)
        df = pd.DataFrame(columns=columns, index=bins)
        group = y.groupby(cate)
        df['箱'] = range(1, len(df)+1)
        df['分数'] = tb.index
        df['样本数量'] = group.size()
        df['关注样本数'] = group.sum()
        df['区间关注类别比例'] = df['关注样本数']/y.sum()
        df['累计关注类别比例'] = df['区间关注类别比例'].cumsum()
        df['区间准确率'] = None
        df['提升'] = None
        df['累计提升'] = None
        return df
    def confusin(self, y, ypred, bin_num=1):
        columns=['实际类别','类别总数','正确百分比','预测类别 非关注','预测类别 关注']
        indexs=['非关注','关注','合计']
        df = pd.DataFrame(columns=columns)
        df['实际类别'] = indexs
        df = df.set_index('实际类别')
        df['类别总数'] = [len(y)-y.sum(), y.sum(), len(y)]
        _, bins = self.get_group(y, ypred)

        y1 = (ypred > bins[bin_num-1]).astype(int)
        df.iloc[:2, -2:] = metrics.confusion_matrix(y, y1)
        df.loc['关注','正确百分比'] = df.at['关注','预测类别 关注']/df.at['关注','类别总数']
        df.loc['非关注','正确百分比'] = df.at['非关注','预测类别 非关注']/df.at['非关注','类别总数']
        df.loc['合计','预测类别 关注'] = df.at['关注','预测类别 关注']/df.loc[:,'预测类别 关注'].sum()
        df.loc['合计','预测类别 非关注'] = df.at['非关注','预测类别 非关注']/df.loc[:,'预测类别 非关注'].sum()
        return df
    def indicate(self, y, ypred, bin_num=1):
        _, bins = self.get_group(y, ypred)
        y1 = (ypred > bins[bin_num-1]).astype(int)
        columns=['名称','正确百分比']
        indexs = ['准确率','精确率','召回率']
        indexs2 = ['灵敏度','特异度','F1统计']
        df1 = pd.DataFrame(columns=columns)
        df1['名称'] = indexs
        df2 = pd.DataFrame(columns=columns)
        df2['名称'] = indexs2
        value = metrics.accuracy_score(y, y1), metrics.precision_score(y, y1), metrics.recall_score(y, y1)
        df1.loc[:,'正确百分比'] = value
        conf = metrics.confusion_matrix(y, y1)
        value = conf[1,1]/conf.sum(axis=1)[1], conf[0,0]/conf.sum(axis=1)[0], metrics.f1_score(y, y1)
        df2.loc[:,'正确百分比'] = value
        return df1, df2

In [198]:
rs = ResultShow()

In [200]:
df1, df2 = rs.indicate(y, ypred,2)

In [201]:
df1

,名称,正确百分比
0,准确率,0.738496
1,精确率,0.811429
2,召回率,0.415205


In [202]:
df2

,名称,正确百分比
0,灵敏度,0.415205
1,特异度,0.939891
2,F1统计,0.549323


In [203]:
rs.confusin(y, ypred,2)

,类别总数,正确百分比,预测类别 非关注,预测类别 关注
实际类别,,,,
非关注,549,0.939891,516,33
关注,342,0.415205,200,142
合计,891,NaN,0.72067,0.811429
